In [69]:
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
#import altair as alt
#from typing import Tuple
#import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
#import mplfinance as fplt
import numpy as np

In [84]:
def six_update_finantial_sheet(dataFrame):
    
    
    simbolos  = list(dataFrame['Ticker'].unique())
    
    
    dataFrame = dataFrame
    
    lista = [
             'sector',
             'industry',
             'ebitdaMargins',
             'profitMargins', 
             'revenueGrowth', 
             'operatingMargins',
             'returnOnAssets',
             'recommendationKey',
             'recommendationMean',
             'debtToEquity', 
             'totalCashPerShare', 
             'quickRatio', 
             'exchange',
             'beta',
             'enterpriseToEbitda', 
             'enterpriseToRevenue', 
             'bookValue', 
             'earningsQuarterlyGrowth', 
             'forwardPE',
             'mostRecentQuarter',
             'logo_url']
    
    for i in lista:
        dataFrame[i] = np.nan
    
    
    for i in simbolos:
        ticker_info = yf.Ticker(str(i))
        info = ticker_info.get_info()
        for l in lista:
            mask = dataFrame['Ticker'] == i
            try:
                dataFrame.loc[mask, l] = info[l]
            except:
                dataFrame.loc[mask, l] = np.nan
    
    #for i in lista:
        #dataFrame[i] = np.nan
        
    
    #for i in lista:
        #dataFrame[i] = dataFrame.apply(lambda x: return_info(x['Symbol'],i),axis=1)
        
    
    return dataFrame

In [71]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [72]:
def euronext_ticker(city, symbol):
    a = symbol
    if city == 'Amsterdam':
        a = symbol + '.AS'
    elif city == 'Brussels':
        a = symbol + '.BR'
    elif city == 'Lisbon':
        a = symbol + '.LS'
    elif city == 'Paris':
        a = symbol + '.PA'
    elif city == 'Oslo':
        a = symbol + '.OL'
    elif city == 'Dublin':
        a = symbol + '.IR'
        
    return a

In [73]:
def num_to_time(date):
    try:
        timestamp = datetime.fromtimestamp(date).strftime('%Y-%m-%d')
    
    except:
        timestamp = "No info"
    
    return timestamp

In [74]:
Amsterdam = '.AS'
Brussels  = '.BR'
Lisbon = '.LS'
Paris = '.PA' 
Oslo = '.OL'
Dublin = '.IR'

In [75]:
df_euro = pd.read_csv('Euronext_Equities_2021-08-15.csv', sep = ';', encoding  = 'ISO-8859-1')

In [76]:
df_euro['City'] = df_euro['Market']

In [77]:
df_euro['City'] = df_euro.apply(lambda x: x['City'].replace("Euronext ", ""),axis=1)
df_euro['City'] = df_euro.apply(lambda x: x['City'].replace("Growth ", ""),axis=1)
df_euro['City'] = df_euro.apply(lambda x: x['City'].replace("Access ", ""),axis=1)
df_euro['City'] = df_euro.apply(lambda x: x['City'].replace(" Børs", ""),axis=1)
df_euro['City'] = df_euro.apply(lambda x: x['City'].replace(" ", ""),axis=1)
df_euro['City'] = df_euro.apply(lambda x: x['City'].replace("Expand", ""),axis=1)

In [85]:
df_euro['Market'] = df_euro.apply(lambda x: x['Market'].replace(" Amsterdam", ""),axis=1)
df_euro['Market'] = df_euro.apply(lambda x: x['Market'].replace(" Brussels", ""),axis=1)
df_euro['Market'] = df_euro.apply(lambda x: x['Market'].replace(" Lisbon", ""),axis=1)
df_euro['Market'] = df_euro.apply(lambda x: x['Market'].replace(" Oslo", ""),axis=1)
df_euro['Market'] = df_euro.apply(lambda x: x['Market'].replace(" Paris", ""),axis=1)
df_euro['Market'] = df_euro.apply(lambda x: x['Market'].replace(" Dublin", ""),axis=1)
df_euro['Market'] = df_euro.apply(lambda x: x['Market'].replace("Oslo ", ""),axis=1)
df_euro['Market'] = df_euro.apply(lambda x: x['Market'].replace(",", ""),axis=1)


In [86]:
df_euro = df_euro.loc[df_euro["Market"] != "Euronext Expert Market"]
df_euro = df_euro.loc[df_euro["Market"] != "Traded not listed Brussels"]

In [87]:
df_euro = explode(df_euro.assign(City=df_euro.City.str.split(',')), 'City')

In [88]:
df_euro['Ticker'] = np.nan

In [89]:
df_euro['Ticker'] = df_euro.apply(lambda x: euronext_ticker(x['City'], x['Symbol']),axis=1)

In [90]:
df_euro.tail(5)

,High,ISIN,Last,Last Date/Time,Low,Market,Name,Open,Symbol,Ticker,...,operatingMargins,profitMargins,quickRatio,recommendationKey,recommendationMean,returnOnAssets,revenueGrowth,sector,totalCashPerShare,City
1692,0.2,BMG9887P1068,0.2,04/08/2015 16:20,0.17,Euronext,ZCI LIMITED,0.17,CV,CV.PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paris
1693,0.1298,CA98936C1068,0.1264,13/08/2021 16:25,0.1234,Euronext Growth,ZENITH ENERGY,0.1298,ZENA,ZENA.OL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oslo
1694,220,BE0003809267,220,06/08/2021 11:30,220,Euronext,ZENOBE GRAMME CERT,220,ZEN,ZEN.BR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brussels
1695,25.82,NO0010721277,24.78,13/08/2021 14:10,24,Euronext Growth,ZWIPE,25.82,ZWIPE,ZWIPE.OL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oslo
1696,11.5,NO0010934748,11.495,13/08/2021 15:05,11.105,Euronext Growth,ØRN SOFTWARE HLD.,11.5,ORN,ORN.OL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oslo


In [91]:
df_euro = six_update_finantial_sheet(df_euro)

In [92]:
for i in list(df_euro.select_dtypes(include='object').columns):
    df_euro[i] = df_euro.apply(lambda x : str(x[i]).replace('ø', '?').replace('Ø', '??'), axis = 1)

In [93]:
df_euro['mostRecentQuarter'] = df_euro.apply(lambda x: num_to_time(x['mostRecentQuarter']) ,axis=1)

In [94]:
df_euro.to_csv('Euronext.csv', encoding = 'UTF-8')

In [95]:
print('i')

i


In [96]:
df_euro

,High,ISIN,Last,Last Date/Time,Low,Market,Name,Open,Symbol,Ticker,...,operatingMargins,profitMargins,quickRatio,recommendationKey,recommendationMean,returnOnAssets,revenueGrowth,sector,totalCashPerShare,City
0,18.5,FR0010285965,18.45,13/08/2021 17:29,18.45,Euronext Growth,1000MERCIS,18.5,ALMIL,ALMIL.PA,...,-0.105100,-0.11319,1.491,none,NaN,-0.04208,-0.099,Communication Services,9.213,Paris
1,117.5,BMG9156K1018,116.5,13/08/2021 16:25,116,B?rs,2020 BULKERS,117,2020,2020.OL,...,0.557460,0.41347,1.167,none,NaN,0.06652,1.630,Industrials,1.019,Oslo
2,4.26,FR0013341781,4.26,13/08/2021 17:36,4.21,Euronext,2CRSI,4.235,2CRSI,2CRSI.PA,...,0.003060,-0.02878,NaN,none,NaN,NaN,NaN,Technology,NaN,Paris
3,9.88,FR0014000T90,9.88,13/08/2021 17:06,9.75,Euronext,2MX ORGANIC,9.784,2MX,2MX.PA,...,0.000000,0.00000,NaN,none,NaN,NaN,NaN,Financial Services,NaN,Paris
4,0.2746,FR0014000TB2,0.2746,09/08/2021 11:38,0.2746,Euronext,2MX ORGANIC BS,0.2746,2MXBS,2MXBS.PA,...,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,Paris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1692,0.2,BMG9887P1068,0.2,04/08/2015 16:20,0.17,Euronext,ZCI LIMITED,0.17,CV,CV.PA,...,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,Paris
1693,0.1298,CA98936C1068,0.1264,13/08/2021 16:25,0.1234,Euronext Growth,ZENITH ENERGY,0.1298,ZENA,ZENA.OL,...,-14.402989,0.00000,0.838,none,NaN,-0.00849,NaN,Energy,0.003,Oslo
1694,220,BE0003809267,220,06/08/2021 11:30,220,Euronext,ZENOBE GRAMME CERT,220,ZEN,ZEN.BR,...,0.534370,0.00264,0.717,none,NaN,0.06408,NaN,Real Estate,NaN,Brussels
1695,25.82,NO0010721277,24.78,13/08/2021 14:10,24,Euronext Growth,ZWIPE,25.82,ZWIPE,ZWIPE.OL,...,-37.024770,0.00000,9.166,none,NaN,-0.30860,-0.322,Technology,3.804,Oslo


0       2020-12-31
1       2021-06-30
2       2021-02-28
3          No info
4          No info
           ...    
1692       No info
1693    2020-09-30
1694       No info
1695    2020-12-31
1696    2021-03-31
Name: mostRecentQuarter, Length: 1697, dtype: object